In [5]:
import pandas as pd
from sqlalchemy import create_engine
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import shutil
import time 

In [6]:
# set executable path and initialize Chrome Browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)


In [7]:
#scrape page into soup
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [8]:
#scrape NASA MARS News
time.sleep(3)
latest_news_article = soup.find("div", class_='list_text')
news_title = latest_news_article.find("div", class_="content_title").text
news_p = latest_news_article.find("div", class_ ="article_teaser_body").text
print(news_title)
print(news_p)

NASA's Mars Perseverance Rover Gets Its Sample Handling System
The system will be collecting and storing Martian rock and soil. Its installation marks another milestone in the march toward the July launch period.


In [9]:
#JPL Featured Space Image
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [10]:
image = soup.find("img", class_="thumb")["src"]
featured_image_url = "https://www.jpl.nasa.gov" + image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23748-640x350.jpg


In [11]:
response = requests.get(featured_image_url, stream=True)
with open('img.jpg', 'wb') as out_file:
   shutil.copyfileobj(response.raw, out_file)


from IPython.display import Image
Image(url='img.jpg')

In [12]:
#Mars Weateher

tweet_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(tweet_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [13]:
#MARS Weather
tweet_url = "https://twitter.com/marswxreport?lang=en"
tweet_html_content = requests.get(tweet_url).text
soup = BeautifulSoup(tweet_html_content, "lxml")
latest_tweet = soup.find_all('div', class_="js-tweet-text-container")


#empty list to hold tweet we are going to keep, used to strip useless content from string
tweets_list = []
# Loop that scans every tweet and searches specifically for those that have weather info
for tweets in latest_tweet: 
    tweet_body = tweets.find('p').text
    if 'InSight' and 'sol' in tweet_body:
        tweets_list.append(tweet_body)
        break
    else: 
        pass
    
mars_weather = ([tweets_list[0]][0][:-26])
tweet_img_link = ([tweets_list[0]][0][-26:])
print(f"{mars_weather}: {tweet_img_link}")

InSight sol 457 (2020-03-10) low -95.7ºC (-140.3ºF) high -9.1ºC (15.6ºF)
winds from the SSE at 6.5 m/s (14.5 mph) gusting to 21.0 m/s (46.9 mph)
pressure at 6.30 hPa: pic.twitter.com/2h0LKoSQrJ


In [14]:
#MARS Facts
mars_facts_url = "https://space-facts.com/mars/"
browser.visit(mars_facts_url)



In [15]:
mars_facts_read_html = pd.read_html(mars_facts_url)
mars_facts_read_html

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [16]:
mars_facts_df = pd.DataFrame(mars_facts_read_html[0])
mars_facts_df.info

<bound method DataFrame.info of                       0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers>

In [17]:
mars_facts_df.columns = ['Mars','Data']
mars_df_table= mars_facts_df.set_index("Mars")
mars_df_table

,Data
Mars,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [18]:
#Mars facts- convert to html
mars_df_table_data = mars_df_table.to_html(classes='marsdata')
mars_df_table_data_html = mars_df_table_data.replace('\n', ' ')
mars_df_table_data_html


'<table border="1" class="dataframe marsdata">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Data</th>     </tr>     <tr>       <th>Mars</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.39 × 10^23 kg (0.11 Earths)</td>     </tr>     <tr>       <th>Moons:</th>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>Orbit Distance:</th>       <td>227,943,824 km (1.38 AU)</td>     </tr>     <tr>       <th>Orbit Period:</th>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>Surface Temperature:</th>       <td>-87 to -5 °C</td>     </tr>     <tr>       <th>First Record:</th>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>Recorded By:</th>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>'

In [19]:
#MARS Hemispheres
mars_hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_hemispheres_url)

In [20]:
hemisphere_image_urls = []

# First, get a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")

# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    browser.back()

C:\Users\nyara\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:536: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


In [21]:
print(hemisphere_image_urls)

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}, {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}, {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}]
